In [1]:
import pandas as pd
import json
import re
import requests
from zhipuai import ZhipuAI

# 初始化API信息
api_key = "f689043de7886e8c604802325fa16392.9m8y9PP2XBKP9ZJy"
access_token = "68e2f9033e3f4236b3916c6658f51fb8"

# 1. 读取数据字典

def load_data_dictionary(file_path):
    excel_file = pd.ExcelFile(file_path)
    first_sheet = excel_file.parse(0).to_json(orient="records", force_ascii=False)
    second_sheet = excel_file.parse(1).to_json(orient="records", force_ascii=False)
    return json.loads(first_sheet), json.loads(second_sheet)

# 2. 读取问题文件
def load_questions(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# 3. 调用ZhipuAI获取相关库表
def find_database_and_table(questions, database_structure):
    client = ZhipuAI(api_key=api_key)
    results = []

    for question_group in questions:
        chat_message = "".join(q['question'] for q in question_group['team'])
        response = client.chat.completions.create(
            model="glm-4-air",
            messages=[
                {"role": "user", "content": f"已知以下数据库 {json.dumps(database_structure)} 确定解决问题 {chat_message} 所需要的库名与表名"},
                {"role": "user", "content": "按照json格式返回 结构与数据库结构一致 只返回json其他的什么都不要逼逼"}
            ]
        )
        results.append({
            "tid": question_group['tid'],
            "team": question_group['team'],
            "related_db_fields": response.choices[0].message.content
        })

    with open("question_to_db_table.json", 'w', encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

    return results

# 4. 匹配字段结构
def match_table_fields(related_db_fields, table_structure):
    for item in related_db_fields:
        item["related_fields"] = [field for field in table_structure if field['table_name'] in item['related_db_fields']]

    with open("db_table_to_fields.json", 'w', encoding="utf-8") as f:
        json.dump(related_db_fields, f, ensure_ascii=False, indent=4)

    return related_db_fields

# 5. 精简表字段信息
def refine_table_fields(related_fields):
    client = ZhipuAI(api_key=api_key)
    refined_results = []

    for item in related_fields:
        response = client.chat.completions.create(
            model="glm-4-air",
            messages=[
                {"role": "user", "content": f"解决问题：{item['history_chat']}需要以下哪些数据库字段支持：{json.dumps(item['related_fields'])}"},
                {"role": "user", "content": "按照数据库字段的json格式返回"}
            ]
        )
        item['refined_fields'] = response.choices[0].message.content
        refined_results.append(item)

    with open("fields_refined.json", 'w', encoding="utf-8") as f:
        json.dump(refined_results, f, ensure_ascii=False, indent=4)

    return refined_results

# 6. 生成SQL语句
def generate_sql(refined_fields):
    client = ZhipuAI(api_key=api_key)
    sql_results = []

    for item in refined_fields:
        response = client.chat.completions.create(
            model="glm-4-air",
            messages=[
                {"role": "user", "content": f"示例sql ：SELECT * FROM constantdb.secumain LIMIT 10 \n已知：{json.dumps(item)}"},
                {"role": "user", "content": "按照sql格式返回"}
            ]
        )
        item['sql'] = response.choices[0].message.content
        sql_results.append(item)

    with open("sql_generated.json", 'w', encoding="utf-8") as f:
        json.dump(sql_results, f, ensure_ascii=False, indent=4)

    return sql_results

# 7. 清洗SQL并获取结果
def execute_sql_and_fetch_results(sql_data):
    pattern = r'(?s)\```sql\n(.*?)\n\```'
    results = []

    for item in sql_data:
        match = re.search(pattern, item.get('sql', ''))
        if match:
            extracted_sql = match.group(1)
            url = "https://comm.chatglm.cn/finglm2/api/query"
            headers = {
                "Content-Type": "application/json",
                "Authorization": f"Bearer {access_token}"
            }
            data = {"sql": extracted_sql, "limit": 10}
            response = requests.post(url, headers=headers, json=data)
            item['query_result'] = response.json()
            results.append(item)

    with open('query_results.json', 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

    return results

# 主程序入口
def main():
    data_file = "data_dictionary.xlsx"
    question_file = "questions.json"

    # 加载数据
    db_structure, table_structure = load_data_dictionary(data_file)
    questions = load_questions(question_file)

    # 执行各阶段处理
    related_db_fields = find_database_and_table(questions, db_structure)
    related_fields = match_table_fields(related_db_fields, table_structure)
    refined_fields = refine_table_fields(related_fields)
    sql_data = generate_sql(refined_fields)
    final_results = execute_sql_and_fetch_results(sql_data)

    print("处理完成，结果已保存。")

if __name__ == "__main__":
    main()


FileNotFoundError: [Errno 2] No such file or directory: 'data_dictionary.xlsx'